In [5]:
import pandas as pd
import numpy as np

In [18]:
#Index column is Incident Date: the first column
dat = pd.read_csv("2merge.csv", index_col=0)

In [19]:
#check data first
dat.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured
0,2019-12-29,Illinois,Danville,1803 block of Westview Ave,0,5
1,2019-12-29,New York,Buffalo,50 block of Henrietta Ave,1,3
2,2019-12-29,California,Ceres,800 block of Allacante Dr,0,5
3,2019-12-27,California,Modesto,1900 block of Vernon Ave,1,3
4,2019-12-27,Georgia,Kennesaw,1575 Ridenour Pkwy NW,0,4


In this file, I'll create two new variables:  

(1) Aggregate address : To get a latitude and longitude from the mapAPI regardless of type. We need to have a new variable with "full address". 

(2) Strength Parameter : We have to variables, %# Killed and %# Injured. Those variables may have different meanings when we analyze, but it is now suitable to draw a time-series graph. We need a parameter that reprents a strength of event.

In [20]:
# Create address : adr
# order: street / city / state
dat['adr']= dat['Address'] + " " + dat['City Or County'] + " " +dat['State']

In [21]:
dat.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,adr
0,2019-12-29,Illinois,Danville,1803 block of Westview Ave,0,5,1803 block of Westview Ave Danville Illinois
1,2019-12-29,New York,Buffalo,50 block of Henrietta Ave,1,3,50 block of Henrietta Ave Buffalo New York
2,2019-12-29,California,Ceres,800 block of Allacante Dr,0,5,800 block of Allacante Dr Ceres California
3,2019-12-27,California,Modesto,1900 block of Vernon Ave,1,3,1900 block of Vernon Ave Modesto California
4,2019-12-27,Georgia,Kennesaw,1575 Ridenour Pkwy NW,0,4,1575 Ridenour Pkwy NW Kennesaw Georgia


In [22]:
# I just found that Incidnet Date is reversed
dat = dat.iloc[::-1]
dat.head()

,Incident Date,State,City Or County,Address,# Killed,# Injured,adr
268,2014-01-01,Virginia,Norfolk,Rockingham Street and Berkley Avenue Extended,2,2,Rockingham Street and Berkley Avenue Extended ...
267,2014-01-03,New York,Queens,Farmers Boulevard and 133rd Avenue,1,3,Farmers Boulevard and 133rd Avenue Queens New ...
266,2014-01-05,Pennsylvania,Erie,829 Parade St,1,3,829 Parade St Erie Pennsylvania
265,2014-01-11,Mississippi,Jackson,3430 W. Capitol Street,0,4,3430 W. Capitol Street Jackson Mississippi
264,2014-01-12,Alabama,Huntsville,University Drive,0,5,University Drive Huntsville Alabama


---

The variables' name is a bit messy. Let's clean up.

* Incident Date -> date
* \# Killed -> n_killed
* \# Injured -> n_injured


---

In [23]:
dat.columns = ['date', 'state', 'city', 'street', 'n_killed', 'n_injured', 'adr']
dat.head()

,date,state,city,street,n_killed,n_injured,adr
268,2014-01-01,Virginia,Norfolk,Rockingham Street and Berkley Avenue Extended,2,2,Rockingham Street and Berkley Avenue Extended ...
267,2014-01-03,New York,Queens,Farmers Boulevard and 133rd Avenue,1,3,Farmers Boulevard and 133rd Avenue Queens New ...
266,2014-01-05,Pennsylvania,Erie,829 Parade St,1,3,829 Parade St Erie Pennsylvania
265,2014-01-11,Mississippi,Jackson,3430 W. Capitol Street,0,4,3430 W. Capitol Street Jackson Mississippi
264,2014-01-12,Alabama,Huntsville,University Drive,0,5,University Drive Huntsville Alabama


---

Create second variable: parameter.

I think about the name but I have nooooo idea, so just say 'p'

Just for now, **p = $N_{killed}^{0.7}+N_{injured}^{0.3}$**

This is arbitrary, but let's update this parameter later.

---

In [24]:
# chage data type
dat = dat.astype({"n_killed": int, "n_injured": int})

In [25]:
dat['p'] = (dat['n_killed']**0.7)+(dat['n_injured']**0.3)

In [26]:
dat.head()

,date,state,city,street,n_killed,n_injured,adr,p
268,2014-01-01,Virginia,Norfolk,Rockingham Street and Berkley Avenue Extended,2,2,Rockingham Street and Berkley Avenue Extended ...,2.855649
267,2014-01-03,New York,Queens,Farmers Boulevard and 133rd Avenue,1,3,Farmers Boulevard and 133rd Avenue Queens New ...,2.390389
266,2014-01-05,Pennsylvania,Erie,829 Parade St,1,3,829 Parade St Erie Pennsylvania,2.390389
265,2014-01-11,Mississippi,Jackson,3430 W. Capitol Street,0,4,3430 W. Capitol Street Jackson Mississippi,1.515717
264,2014-01-12,Alabama,Huntsville,University Drive,0,5,University Drive Huntsville Alabama,1.620657


---

Now I don't like the column ordering. 

Let's clean up.

---

In [27]:
dat = dat[['date', 'adr', 'p', 'n_killed', 'n_injured', 'street', 'city', 'state']]

In [28]:
dat.head()

,date,adr,p,n_killed,n_injured,street,city,state
268,2014-01-01,Rockingham Street and Berkley Avenue Extended ...,2.855649,2,2,Rockingham Street and Berkley Avenue Extended,Norfolk,Virginia
267,2014-01-03,Farmers Boulevard and 133rd Avenue Queens New ...,2.390389,1,3,Farmers Boulevard and 133rd Avenue,Queens,New York
266,2014-01-05,829 Parade St Erie Pennsylvania,2.390389,1,3,829 Parade St,Erie,Pennsylvania
265,2014-01-11,3430 W. Capitol Street Jackson Mississippi,1.515717,0,4,3430 W. Capitol Street,Jackson,Mississippi
264,2014-01-12,University Drive Huntsville Alabama,1.620657,0,5,University Drive,Huntsville,Alabama


In [29]:
dat.to_csv("3new-var.csv", mode = "w")